# Librerias

In [1]:
import numpy as np
import math
import sys
import random

## Simulation parameters

In [2]:
Number_Reps =10
Initial_SimTime =30
scvX = 0.9
rho1X = 0

## Rate Function

In [3]:
def NewGetMMECO(n_reps,initial_simtime,scvX,rho1X):
    TwoMomsOrder=[]
    ImpThirdMoms=0
    IntNote = 10
    StepSize = 0.01
    MaxMult = 10

    if initial_simtime/StepSize+1 > 15000:
        StepSize = StepSize*10
    if IntNote < n_reps/2:
        IntNote = np.floor(n_reps/2)
    elif IntNote < n_reps/200:
        IntNote = np.floor(n_reps/200) # o int()
    NumberofSteps = (initial_simtime/StepSize)+1
    
    MeanSvTime = 1
    TrueVarMoms = scvX*(MeanSvTime**2) # Acá obtengo la varianza
    TrueSecMoms = TrueVarMoms+pow(MeanSvTime,2) # y acá el segundo momento de la variaza(Var[X]+E[X]^2)
    Coeffvar = np.sqrt(scvX) # Determino el coeficiente de variación
    
    if scvX == 1:
        # exponential
        TwoMomsOrder = 1
        ImpThirdMoms = 6 # E[X^3]=3!*(1/lambda)^3, con lambda=1 da 6
    elif scvX<1:
        #MECO (Mixture of Erlang of Common Order)
        k=1
        while 1/k>scvX:
            k+=1
        #Parametros    
        MEConOrder = k
        MEConAlpha = (1/(1+scvX))*(k*scvX - np.sqrt(k*(1+scvX)-(k**2)*scvX))
        MEConRate = (k-MEConAlpha)/MeanSvTime
        SteadyVector = []
        SteadyCumVector = []
        for i in range(2*k-1):
            SteadyVector.append(MEConAlpha/(k-1+MEConAlpha))
        for i in range(k+1,2*k-1):
            SteadyVector.append((1-MEConAlpha)/(k-1+MEConAlpha))
        
        TwoMomsOrder = 2*MEConOrder-1
        TMat = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        TMatInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        TMatSqInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))        
        TMatCuInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))

        
        ZetaVec = [0 for x in range(TwoMomsOrder)]
        ZetaVec[0] = 1-MEConAlpha
        ZetaVec[MEConOrder] = MEConAlpha
        for i in range(MEConOrder):
            for j in range(MEConOrder):
                TMatInv[i][j] = -1/MEConRate 
        for i in range(MEConOrder+1,2*MEConOrder-1):
            for j in range(i,2*MEConOrder-1):
                TMatInv[i][j] = -1/MEConRate         



    else:
        # Hyperexponencial
        TwoMomsOrder = 2
        TMat = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        TMatInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        TMatCuInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        TMatSqInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        ZetaVec = [0 for x in range(TwoMomsOrder)]
        SteadyVector = []
        for i in range(TwoMomsOrder):
            SteadyVector.append(1/TwoMomsOrder)
            
        h2bAlpha = 0.5*(1+np.sqrt(1-2/(scvX+1)))
        h2bRate = 2*h2bAlpha/MeanSvTime
        
        TMat[0,0] = -h2bRate
        TMat[1,1] = -h2bRate*(1-h2bAlpha)/h2bAlpha
        ZetaVec[0] = h2bAlpha
        ZetaVec[1] = 1-h2bAlpha
        for i in range(TwoMomsOrder):
            TMatInv[i][i] = 1/TMat[i][i]
            
    if scvX>1 or scvX<1:
        #Se calcula el tercer momento :s
        for i in range(TwoMomsOrder):
            for j in range(TwoMomsOrder):
                for k in range(TwoMomsOrder):
                    TMatSqInv[i][j] = TMatSqInv[i][j]+TMatInv[i][k]*TMatInv[k][j]
                    
        for i in range(TwoMomsOrder):
            for j in range(TwoMomsOrder):
                for k in range(TwoMomsOrder):
                    TMatCuInv[i][j] = TMatCuInv[i][j]+TMatInv[i][k]*TMatSqInv[k][j]
        
        for i in range(TwoMomsOrder):
            for j in range(TwoMomsOrder):
                ImpThirdMoms = ImpThirdMoms-6*ZetaVec[i]*TMatCuInv[i][j]
    
    #Se busca el markov MECO que calze
    SkewX = (ImpThirdMoms-3*MeanSvTime*TrueSecMoms+2*pow(MeanSvTime,3))/(pow(TrueVarMoms,1.5))
    MajPhType = 5 #Este es el parametro que dijo el profe

    if Coeffvar>0 and SkewX>= (Coeffvar- (1/Coeffvar)):
        MinN1 = 1/scvX
        MinN2 = (-SkewX+1/Coeffvar**3+1/Coeffvar+2*Coeffvar)/(SkewX- (Coeffvar-1/Coeffvar))
        
        #chequea factibilidad pero no entiendo cómo?
        if MinN1>MinN2:
            k = int(MinN1)+1
        else:
            k = int(MinN2)+1 
    else:
        print("Bad MECO")
        sys.exit(0)
        

    NeedBiggerOrder = True
    while NeedBiggerOrder == True:
        mecoM1 = MeanSvTime
        mecoM2 = TrueSecMoms
        mecoM3 = ImpThirdMoms
        
        mecoX = mecoM1*mecoM3-((k+2)/(k+1))*(mecoM2**2)
        mecoY = mecoM2-((k+1)/k)*(mecoM1**2)
        mecoA = k*(k+2)*mecoM1*mecoY
        mecoB = -(k*mecoX+k*((k+2)/(k+1))*(mecoY**2)+(k+2)*mecoY*(mecoM1**2))
        mecoC = mecoM1*mecoX

        mecoRoot1 = (-mecoB + math.sqrt((mecoB**2) - 4 * mecoA * mecoC)) / (2 * mecoA)
        mecoRoot2 = (-mecoB - math.sqrt((mecoB**2) - 4 * mecoA * mecoC)) / (2 * mecoA)
        
        MECOOrder = k
        MECORate1 = 1/mecoRoot1
        MECORate2 = 1/mecoRoot2
        MECOAlpha = ((mecoM1/MECOOrder)-mecoRoot2)/(mecoRoot1-mecoRoot2)
        
        # A continuacion trabaja con el MArkovMeco
        tempMMECOCov = TrueVarMoms*rho1X
        MMECOAlpha21 = MECOAlpha-tempMMECOCov/((1-MECOAlpha)*((MECOOrder*mecoRoot1-MECOOrder*mecoRoot2)**2))
        MMECOAlpha12 = MMECOAlpha21*(1-MECOAlpha)/MECOAlpha
        MMECORate1 = MECORate1
        MMECORate2 = MECORate2
        
        #Chequea factibilidad
        if MMECOAlpha21<0 or MMECOAlpha21>1 or MMECOAlpha12<0 or MMECOAlpha12>1:
            k += 1
            if k>4000:
                print("No se pueden unir los momentos,chequear un valor factible para rho1....")
                sys.exit(0)
                
        else:
            NeedBiggerOrder = False
            MMECOOrder = MECOOrder         
    SteadyVector = []
    
    for i in range(MMECOOrder):
        SteadyVector.append((MECORate2*MMECOAlpha21)/(MMECOOrder*(MECORate1*MMECOAlpha12+MECORate2*MMECOAlpha21)))
    
    for i in range(MMECOOrder+1,2*MMECOOrder):
        SteadyVector.append((MECORate1*MMECOAlpha12)/(MMECOOrder*(MECORate1*MMECOAlpha12+MECORate2))) 
    
    MMECOCInf = scvX*(1+2*MECOAlpha*rho1X/MMECOAlpha21)
    
    SteadyCumVector = [SteadyVector[0]]
    l = SteadyCumVector[0]
    for i in range(1,len(SteadyVector)):
        SteadyCumVector.append(SteadyVector[i]+SteadyCumVector[i-1])
#         print(SteadyCumVector)
    return (ImpThirdMoms,SteadyCumVector,MMECOOrder,MMECORate1,MMECORate2,MMECOAlpha12,MMECOAlpha21)

In [4]:
# SteadyCumVector=[]
# ImpThirdMoms,SteadyCumVector,MMECOOrder,MMECORate1,MMECORate2,MMECOAlpha12,MMECOAlpha21=NewGetMMECO(n_reps=100,initial_simtime=10,scvX=1.5,rho1X=0.5)
# print(ImpThirdMoms)
# print(SteadyCumVector)
# print(MMECOOrder)
# print(MMECORate1)
# print(MMECORate2)
# print(MMECOAlpha12)
# print(MMECOAlpha21)

## First Potential Arrival

In [5]:
def ErlangOneStep(n,Beta):
    Prod=0.01
    for i in range(n): 
        U = random.uniform(0, 1) 
        Prod = Prod * (1 - U)
#         print(Prod)
    ErlangOneStep = -Beta * math.log(Prod)
    return(ErlangOneStep)

In [6]:
def first_potential_arrival(MajPhType,MMECOOrder,MMECORate1,MMECORate2):
    if MajPhType == 5:
        Run=True
        while Run==True:
            WhichBasePh = random.uniform(0, 1)
            flag=False
            for i in range(len(SteadyCumVector)):
                if WhichBasePh>SteadyCumVector[i]:
                    flag=True     
            if flag==True:
                R = 0
                while WhichBasePh < SteadyCumVector[R]:
                    R +=1
                CurPhase = R
                if CurPhase < MMECOOrder +1:
                    FirstArrTime = ErlangOneStep(n=MMECOOrder+1-CurPhase,Beta=(1/MMECORate1))
                    PrevGenErlang = 1
                    Run=False                       
                else:
                    FirstArrTime = ErlangOneStep(n=MMECOOrder+1-CurPhase,Beta=1/MMECORate2)
                    PrevGenErlang = 2   
                Run=False    
    return(FirstArrTime,PrevGenErlang)                      

In [7]:
# for i in range(100):
#     FirstArrTime,PrevGenErlang=first_potential_arrival(MajPhType=5,MMECOOrder=MMECOOrder,MMECORate1=MMECORate1,MMECORate2=MMECORate2)
#     print("FirstArrTime= ", FirstArrTime)

# MakeArrival

## InvRateFunction

In [8]:
def MyInvRateFunc(CurValue,EndTime,EqualSpace,Intervals,valores):
    NumInts = len(Intervals)+1
    IntVals=[0 for i in range(NumInts)]
    FuncVals=[0 for i in range(NumInts)]
    IntEndPts=[0 for i in range(NumInts)]
    IntEndPts[0] = 0
    IntEndPts[NumInts-1] = EndTime
    FuncVals[0] = 0
    
    if EqualSpace == True: 
        for IntPt in range(NumInts):
            if IntPt != 0:
                IntEndPts[IntPt] = IntEndPts[IntPt - 1] + EndTime / NumInts

    for i in range(NumInts):
        IntVals[i] =valores[i]
    
    for IntPt in range(NumInts):
        if IntPt != 0:
            FuncVals[IntPt] = FuncVals[IntPt - 1] + IntVals[IntPt] * (IntEndPts[IntPt] - IntEndPts[IntPt - 1])
            
    IntPt = 0
    while CurValue > FuncVals[IntPt]:
        IntPt = IntPt + 1
    CurInt = IntPt
    InvTime = IntEndPts[CurInt - 1] + (CurValue - FuncVals[CurInt - 1]) / IntVals[CurInt]
    return(InvTime)

In [9]:
# InvTime=MyInvRateFunc(CurValue=15,EndTime=30,EqualSpace=True,Intervals=[3,6,9,12,15,18,21,24,27],valores=[6,1.85,0.8,2.25,5.65,2.55,7.15,7.35,3.05,6.7])
# print(InvTime)

## CumRateFunction

In [10]:
def MyCumRateFunc(CurTime,EndTime,EqualSpace,Intervals,valores):
    NumInts = len(Intervals)+1
    IntVals=[0 for i in range(NumInts)]
    FuncVals=[0 for i in range(NumInts)]
    IntEndPts=[0 for i in range(NumInts)]
    IntEndPts[0] = 0
    IntEndPts[NumInts-1] = EndTime
    FuncVals[0] = 0
    
    if EqualSpace == True: 
        for IntPt in range(NumInts):
            if IntPt != 0:
                IntEndPts[IntPt] = IntEndPts[IntPt - 1] + EndTime / NumInts

    for i in range(NumInts):
        IntVals[i] =valores[i]
    
    for IntPt in range(NumInts):
        if IntPt != 0:
            FuncVals[IntPt] = FuncVals[IntPt - 1] + IntVals[IntPt] * (IntEndPts[IntPt] - IntEndPts[IntPt - 1])
            
    IntPt = 0
    while CurTime > IntEndPts[IntPt]:
        IntPt = IntPt + 1
    CurInt = IntPt
    CheckValue = FuncVals[CurInt-1]+IntVals[CurInt]* (CurTime - IntEndPts[CurInt-1])
    return(CheckValue)

In [11]:
# CheckValue=MyCumRateFunc(CurTime=10,EndTime=30,EqualSpace=True,Intervals=[3,6,9,12,15,18,21,24,27],valores=[6,1.85,0.8,2.25,5.65,2.55,7.15,7.35,3.05,6.7])
# print(CheckValue)

## MakeArrivalFunction

In [12]:
def MakeArrival(MajPhType,PrevGenErlang,MMECOAlpha12,MMECOAlpha21):
    if MajPhType == 5:
        WhichBasePh =random.uniform(0, 1)
        if PrevGenErlang == 1:  
            if WhichBasePh < (1 - MMECOAlpha12): 
                NextArrTime = ErlangOneStep(n=MMECOOrder,Beta=(1/MMECORate1))
                PrevGenErlang = 1
            else:
                NextArrTime = ErlangOneStep(n=MMECOOrder,Beta=(1/MMECORate2))
                PrevGenErlang = 2
        else:
            if WhichBasePh < MMECOAlpha21: 
                NextArrTime =ErlangOneStep(n=MMECOOrder,Beta=(1/MMECORate1))
                PrevGenErlang = 1
            else:
                NextArrTime = ErlangOneStep(n=MMECOOrder,Beta=(1/MMECORate2)) 
                PrevGenErlang = 2
    
    CurArrTime = MyCumRateFunc(CurTime=10,EndTime=30,EqualSpace=True,Intervals=[3,6,9,12,15,18,21,24,27],valores=[6,1.85,0.8,2.25,5.65,2.55,7.15,7.35,3.05,6.7]) 
    
    if (CurArrTime + NextArrTime) > LastS: 
        InvTime = Initial_SimTime + 1
    else:
        InvTime = MyInvRateFunc(CurValue=15,EndTime=30,EqualSpace=True,Intervals=[3,6,9,12,15,18,21,24,27],valores=[6,1.85,0.8,2.25,5.65,2.55,7.15,7.35,3.05,6.7]) 

    
    
    return(NextArrTime,PrevGenErlang)              


In [13]:
# LastS=0
# NextArrTime,PrevGenErlang=MakeArrival(MajPhType=5,PrevGenErlang=PrevGenErlang,MMECOAlpha12=MMECOAlpha12,MMECOAlpha21=MMECOAlpha21)
# print("NextArrTime= ",NextArrTime)

# GetRho

In [14]:
def Get_rho_range(scvX,ImpThirdMoms):
    MaxOrderSize = 50000 
    CanMakeMMECO = True
    MeanSvTime = 1 
    TrueVarMoms = scvX * (MeanSvTime**2)
    TrueSecMoms = TrueVarMoms + (MeanSvTime**2)
    Coeffvar = math.sqrt(scvX)
    
    
    if scvX == 1:
        #Exponencial
        TwoMomsOrder = 1 
        ImpThirdMoms = 6
    elif scvX < 1:
        #MECon
        k = 1 
        while (1 / k) > scvX: 
            k = k + 1

        MEConOrder = k 
        MEConAlpha = (1 / (1 + scvX)) * (k * scvX - math.sqrt(k * (1 + scvX) - (k**2) * scvX)) 
        MEConRate = (k - MEConAlpha) / MeanSvTime
        
        TwoMomsOrder = 2 * MEConOrder - 1 
        TMat = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        TMatInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        TMatSqInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))        
        TMatCuInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        ZetaVec = [0 for x in range(TwoMomsOrder)]
        ZetaVec[0] = 1-MEConAlpha
        ZetaVec[MEConOrder] = MEConAlpha
        
        for i in range(TwoMomsOrder): 
            for j in range(TwoMomsOrder): 
                TMatInv[i][j] = -1/MEConRate 
                
        for i in range(MEConOrder+1,2*MEConOrder-1):
            for j in range(i,2*MEConOrder-1):
                TMatInv[i][j] = -1/MEConRate 
 
    else:
        #h2b
        TwoMomsOrder = 2 
        TMat = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        TMatInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        TMatSqInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))        
        TMatCuInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        ZetaVec = [0 for x in range(TwoMomsOrder)]
        h2bAlpha = 0.5*(1+np.sqrt(1-2/(scvX+1)))
        h2bRate = 2*h2bAlpha/MeanSvTime
        TMat[0][0] = -h2bRate
        TMat[1][1] = -h2bRate*(1-h2bAlpha)/h2bAlpha
        ZetaVec[0] = h2bAlpha
        ZetaVec[1] = 1-h2bAlpha
        for i in range(TwoMomsOrder):
            TMatInv[i][i] = 1/TMat[i][i]        
        
    if scvX > 1 or scvX < 1:
        for i in range(TwoMomsOrder):
            for j in range(TwoMomsOrder):
                for k in range(TwoMomsOrder):
                    TMatSqInv[i][j] = TMatSqInv[i][j]+TMatInv[i][k]*TMatInv[k][j]
                    
        for i in range(TwoMomsOrder):
            for j in range(TwoMomsOrder):
                for k in range(TwoMomsOrder):
                    TMatCuInv[i][j] = TMatCuInv[i][j]+TMatInv[i][k]*TMatSqInv[k][j]
        for i in range(TwoMomsOrder):
            for j in range(TwoMomsOrder):
                ImpThirdMoms = ImpThirdMoms-6*ZetaVec[i]*TMatCuInv[i][j]

    SkewX = (ImpThirdMoms-3*MeanSvTime*TrueSecMoms+2*(MeanSvTime**3))/(TrueVarMoms**(3/2))
    
    
    if Coeffvar > 0 and SkewX >= (Coeffvar-(1/Coeffvar)):
        MinN1 = 1/scvX
        MinN2 = (-SkewX+1/(Coeffvar**3)+1/Coeffvar+2*Coeffvar)/(SkewX-(Coeffvar-1/Coeffvar))
        if MinN1>MinN2:
            k = int(MinN1)+1
        else:
            k = int(MinN2)+1 
    else:
        print("Bad MECO")
        sys.exit(0)
    
    if MinN1 > MinN2:
        InitK = int(MinN1)+1
    else:
        InitK = int(MinN2)+1
        
    CurMaxGuess = 1
    FoundMax = False
    
    #Busco el max rho
    k = InitK
    while FoundMax == False:
        NeedBiggerOrder = True
        while NeedBiggerOrder == True:
                mecoM1 = MeanSvTime
                mecoM2 = TrueSecMoms
                mecoM3 = ImpThirdMoms
                mecoX = mecoM1*mecoM3-((k+2)/(k+1))*(mecoM2**2)
                mecoY = mecoM2-((k+1)/k)*(mecoM1**2)
                mecoA = k*(k+2)*mecoM1*mecoY
                mecoB = -(k*mecoX+k*((k+2)/(k+1))*(mecoY**2)+(k+2)*mecoY*(mecoM1**2))
                mecoC = mecoM1*mecoX
                mecoRoot1 = (-mecoB + math.sqrt((mecoB**2) - 4 * mecoA * mecoC)) / (2 * mecoA)
                mecoRoot2 = (-mecoB - math.sqrt((mecoB**2) - 4 * mecoA * mecoC)) / (2 * mecoA)
                TempMECOOrder = k
                MECORate1 = 1/mecoRoot1
                MECORate2 = 1/mecoRoot2
                MECOAlpha = ((mecoM1/TempMECOOrder)-mecoRoot2)/(mecoRoot1-mecoRoot2)
                
                #Marjov MECO stuff
                tempMMECOCov = TrueVarMoms*CurMaxGuess
                MMECOAlpha21 = MECOAlpha-tempMMECOCov/((1-MECOAlpha)*((TempMECOOrder*mecoRoot1-TempMECOOrder*mecoRoot2)**2))
                MMECOAlpha12 = MMECOAlpha21*(1-MECOAlpha)/MECOAlpha
                MMECORate1 = MECORate1
                MMECORate2 = MECORate2    
                # Chequea factibilidad
                if MMECOAlpha21 < 0 or MMECOAlpha21 > 1 or MMECOAlpha12 < 0 or MMECOAlpha12 > 1:
                    k=k+1
                    if k > MaxOrderSize:
                        NeedBiggerOrder=False
                        CurMaxGuess = CurMaxGuess-0.001
                        k = InitK
                    else:
                        NeedBiggerOrder = False
                        FoundMax = True
    
    #Busco el menor rho 
    MaxRho1 = CurMaxGuess
    CurMinGuess = MaxRho1
    FoundMin = False
    k = InitK
    while FoundMin == False:
        NeedBiggerOrder = True
        while NeedBiggerOrder == True:
            mecoM1 = MeanSvTime
            mecoM2 = TrueSecMoms
            mecoM3 = ImpThirdMoms
            mecoX = mecoM1*mecoM3-((k+2)/(k+1))*(mecoM2**2)
            mecoY = mecoM2-((k+1)/k)*(mecoM1**2)
            mecoA = k*(k+2)*mecoM1*mecoY
            mecoB = -(k*mecoX+k*((k+2)/(k+1))*(mecoY**2)+(k+2)*mecoY*(mecoM1**2))
            mecoC = mecoM1*mecoX
            mecoRoot1= (-mecoB + math.sqrt((mecoB**2) - 4 * mecoA * mecoC)) / (2 * mecoA)
            mecoRoot2= (-mecoB - math.sqrt((mecoB**2) - 4 * mecoA * mecoC)) / (2 * mecoA) 
            TempMECOOrder = k
            MECORate1 = 1/mecoRoot1
            MECORate2 = 1/mecoRoot2
            MECOAlpha = ((mecoM1/TempMECOOrder)-mecoRoot2)/(mecoRoot1-mecoRoot2)
            tempMMECOCov = TrueVarMoms*CurMinGuess
            MMECOAlpha21 = MECOAlpha-tempMMECOCov/((1-MECOAlpha)*((TempMECOOrder*mecoRoot1-TempMECOOrder*mecoRoot2)**2))
            MMECOAlpha12 = MMECOAlpha21*(1-MECOAlpha)/MECOAlpha
            MMECORate1 = MECORate1
            MMECORate2 = MECORate2        
            #chequea factibilidad
            if MMECOAlpha21 < 0 or MMECOAlpha21 > 1 or MMECOAlpha12 < 0 or MMECOAlpha12 > 1:
                k += 1
                if k>MaxOrderSize:
                    FoundMin = True
                    NeedBiggerOrder=False
                else:
                    NeedBiggerOrder = False
                    CurMinGuess = CurMinGuess-0.001
                    k = InitK
    MinRho1 = CurMinGuess
    return(MinRho1,MaxRho1)   

In [15]:
# MinRho1,MaxRho1=Get_rho_range(scvX=0.5,ImpThirdMoms=1)
# print("MinRho1= ",MinRho1)
# print("MaxRho1= ",MaxRho1)

# Main

In [16]:
LastS=0
Tiempo_simulacion=100
Repeticiones=10
for i in range(Repeticiones):
    print("Inicia Repeticion",i)
    SteadyCumVector=[]
    Tiempo=0
    ImpThirdMoms,SteadyCumVector,MMECOOrder,MMECORate1,MMECORate2,MMECOAlpha12,MMECOAlpha21=NewGetMMECO(n_reps=100,initial_simtime=10,scvX=1.5,rho1X=0.5)    
    FirstArrTime,PrevGenErlang=first_potential_arrival(MajPhType=5,MMECOOrder=MMECOOrder,MMECORate1=MMECORate1,MMECORate2=MMECORate2)
    Tiempo+=FirstArrTime
    while Tiempo<Tiempo_simulacion:
        NextArrTime,PrevGenErlang=MakeArrival(MajPhType=5,PrevGenErlang=PrevGenErlang,MMECOAlpha12=MMECOAlpha12,MMECOAlpha21=MMECOAlpha21)
        print("NextArrTime= ",NextArrTime)
        Tiempo+=NextArrTime
    print("\n")    

Inicia Repeticion 0
NextArrTime=  9.383889650966447
NextArrTime=  6.262272151315761
NextArrTime=  8.5393575846462
NextArrTime=  9.167897867240598
NextArrTime=  8.495428555370044
NextArrTime=  7.646332677139089
NextArrTime=  5.908448996733048
NextArrTime=  7.454510231743968
NextArrTime=  9.50953932096231
NextArrTime=  7.958472762621724
NextArrTime=  6.8358803519339695
NextArrTime=  8.671736628405773


Inicia Repeticion 1
NextArrTime=  9.009172634356464
NextArrTime=  7.5938231600540345
NextArrTime=  5.9276052628541755
NextArrTime=  10.09245406962306
NextArrTime=  10.366175017806844
NextArrTime=  6.253739357273746
NextArrTime=  7.766983095775268
NextArrTime=  7.88360306611346
NextArrTime=  1.3698193127187865
NextArrTime=  5.412440237865018
NextArrTime=  6.463672638404468
NextArrTime=  1.1643951590604942
NextArrTime=  1.0481627235888902
NextArrTime=  8.31881855276813
NextArrTime=  5.936211308492423


Inicia Repeticion 2
NextArrTime=  5.312402727428031
NextArrTime=  6.504047614684979
NextAr